# Data mining

# Lesson 6

# Graph Neural Networks (GNN) and Reinforcement Learning (RL)

### **Objectives:**
- Understand the basics of Graph Neural Networks (GNNs) and their applications.
- Explore Reinforcement Learning (RL) concepts such as agents, rewards, and policies.
- Implement GNNs for node classification on synthetic graph data.
- Use RL to solve a graph traversal problem.

### **Description**

Graph Neural Networks (GNNs) and Reinforcement Learning (RL) are emerging tools in modern machine learning. GNNs enable effective learning on graph-structured data, such as social networks, molecular structures, or transportation systems, while RL allows for decision-making through rewards in dynamic environments.

In this lab:

1) We will simulate graph data and train a Graph Neural Network (GNN) to perform node classification.
2) We will integrate Reinforcement Learning to train an agent to traverse a graph while maximizing cumulative rewards.


### Libraries that we use:

- [Torch](https://pytorch.org/) - for GNN implementation.
- [Matplotlib](https://matplotlib.org/) and [Seaborn](https://seaborn.pydata.org/) - for data visualization and identifying interesting patterns.
- [networkx](https://networkx.org/) - for graph generation and visualization.
- [gym](https://github.com/openai/gym) - for Reinforcement Learning environment.


#### Structure: Synthetic graph dataset.

We will simulate a random graph with labeled nodes for a node classification task. Each node will belong to one of N classes.

- Generate a graph with networkx.
- Add synthetic node features and labels.

In [ ]:
import networkx as nx
import numpy as np
import torch
from torch_geometric.data import Data
import matplotlib.pyplot as plt

# Parameters
n_nodes = 100    # Number of nodes
n_edges = 200    # Number of edges
n_classes = 3    # Number of node classes
n_features = 5   # Number of node features

# Generate a random graph
G = nx.gnm_random_graph(n_nodes, n_edges, seed=42)

# Assign random features to nodes
node_features = torch.randn((n_nodes, n_features))

# Assign random labels (classes) to nodes
node_labels = torch.randint(0, n_classes, (n_nodes,))

# Convert edges to tensor
edge_index = torch.tensor(list(G.edges)).t().contiguous()

# Create a PyTorch Geometric data object
data = Data(x=node_features, edge_index=edge_index, y=node_labels)

# Visualize the graph
plt.figure(figsize=(8, 6))
nx.draw(G, with_labels=True, node_color=node_labels, cmap='coolwarm', node_size=100)
plt.title("Synthetic Graph with Node Labels")
plt.show()


## **Exercise 1:** Implement a Graph Neural Network (GNN)
- Create a GNN model using the torch_geometric library.
- Train the GNN to classify the nodes based on their features.
- Evaluate its performance.

In [ ]:
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.loader import DataLoader

# Define a simple GNN model
class GNN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GNN, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

# Model, optimizer, and loss
model = GNN(in_channels=n_features, hidden_channels=16, out_channels=n_classes)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

# Train the GNN model
def train():
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = criterion(out, data.y)
    loss.backward()
    optimizer.step()
    return loss.item()

# Evaluate the GNN model
@torch.no_grad()
def test():
    model.eval()
    pred = model(data.x, data.edge_index).argmax(dim=1)
    acc = (pred == data.y).sum().item() / n_nodes
    return acc

# Training loop
for epoch in range(200):
    loss = train()
    acc = test()
    if epoch % 20 == 0:
        print(f"Epoch {epoch}, Loss: {loss:.4f}, Accuracy: {acc:.4f}")


## **Exercise 2:** Reinforcement Learning on a Graph Traversal Problem
- We will design a Reinforcement Learning environment where an agent navigates through the graph to maximize cumulative rewards.
1. Define a graph environment: Nodes are states, edges are actions, and rewards are provided for specific nodes.
2. Train an RL agent to navigate the graph.

#### 2.1 Create a Custom Graph Environment Using OpenAI Gym

In [ ]:
import gym
from gym import spaces

class GraphTraversalEnv(gym.Env):
    def __init__(self, graph, start_node, goal_node):
        super(GraphTraversalEnv, self).__init__()
        self.graph = graph
        self.start_node = start_node
        self.goal_node = goal_node
        self.current_node = start_node

        # Action and observation space
        self.action_space = spaces.Discrete(len(graph.nodes))  # Move to any node
        self.observation_space = spaces.Discrete(len(graph.nodes))

    def reset(self):
        self.current_node = self.start_node
        return self.current_node

    def step(self, action):
        if (self.current_node, action) in self.graph.edges:
            self.current_node = action

        # Reward: +10 for reaching goal, -1 otherwise
        reward = 10 if self.current_node == self.goal_node else -1
        done = self.current_node == self.goal_node

        return self.current_node, reward, done, {}

    def render(self, mode='human'):
        print(f"Current Node: {self.current_node}")


#### 2.2 Train a Q-Learning Agent

In [ ]:
from collections import defaultdict
import random

# Initialize environment
start_node = 0
goal_node = n_nodes - 1
env = GraphTraversalEnv(G, start_node, goal_node)

# Q-Learning parameters
Q = defaultdict(lambda: np.zeros(len(G.nodes)))
alpha = 0.1
gamma = 0.9
epsilon = 0.2
episodes = 500

# Q-Learning algorithm
for episode in range(episodes):
    state = env.reset()
    done = False
    while not done:
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample()  # Explore
        else:
            action = np.argmax(Q[state])  # Exploit

        next_state, reward, done, _ = env.step(action)
        Q[state][action] += alpha * (reward + gamma * np.max(Q[next_state]) - Q[state][action])
        state = next_state

    if episode % 50 == 0:
        print(f"Episode {episode}, Reward: {reward}")


## **Exercise 3:** Visualize the Agent’s Path
- Simulate the trained agent navigating the graph.
- Plot the agent’s path from the start node to the goal node.

In [ ]:
# Simulate agent's path
state = env.reset()
path = [state]
done = False

while not done:
    action = np.argmax(Q[state])
    state, _, done, _ = env.step(action)
    path.append(state)

print(f"Agent's Path: {path}")

# Visualize the path on the graph
plt.figure(figsize=(8, 6))
node_colors = ['green' if i in path else 'lightblue' for i in range(n_nodes)]
nx.draw(G, with_labels=True, node_color=node_colors, node_size=300)
plt.title("Agent's Path Through the Graph")
plt.show()


## Consclusion:

We learned: 

- Understand the basics of Graph Neural Networks (GNNs) and their applications.
- Explore Reinforcement Learning (RL) concepts such as agents, rewards, and policies.
- Implement GNNs for node classification on synthetic graph data.
- Use RL to solve a graph traversal problem.

This lab introduces students to GNNs for node classification and RL for graph traversal, providing a solid foundation for graph-based learning tasks.


